# Problem description
On the files attached you can find different information about the population in
various counties in the USA and how it changed over the years including size age,
income, race, age.
The situation
The year is 2021.
You have your own business, a consulting firm specialized in location intelligence
marketing. In other words, you advise new and existing business franchises where
they should open new locations based on various informations about the locations.
With the information in hand, answere the questions to the following business
situations:

## Client 1: 
Your client #1 has several Senior Living Communities in the USA and is
looking for the next 3 locations (Counties) to open.
The client's clientele is typically a mid-income senior.
Which locations would you recommend and why? Justify your choice.

To recommend the top 3 counties for opening Senior Living Communities, we need to consider the following factors:

1. Population by age: We need to identify counties with a higher percentage of the senior population (65 years and older) as they are the target audience for the client's Senior Living Communities.

2. Income: The client is looking for mid-income seniors, so we need to consider counties with a median income that is close to the national average.

Based on the above factors, we can use the 'Population by Age and Sex - US, States, Counties.csv' and 'BEA - US, States, Counties - Personal Income.csv' data files to recommend the top 3 counties for opening Senior Living Communities.

In [30]:
import pandas as pd

# Load population by age data
population_age_df = pd.read_csv('Population by Age and Sex - US, States, Counties.csv')
population_age_df = population_age_df[population_age_df['Year'] == 2019] # Filter data for year 2019
population_age_df = population_age_df[['IBRC_Geo_ID', 'Description', 'Population 65+', 'Total Population']] # Select relevant columns
population_age_df['County'] = population_age_df['Description'].str.split(',').str[0] # Extract county name from description column
population_age_df['State'] = population_age_df['Description'].str.split(',').str[1].str.strip() # Extract state name from description column
population_age_df.drop('Description', axis=1, inplace=True) # Drop description column

# Load personal income data
income_df = pd.read_csv('/content/drive/MyDrive/Documents/5.Work/6. UrDoer - 6 Mar 2023/5.2300 - Programming for Data Science/data/BEA - US, States, Counties - Personal Income.csv')
income_df = income_df[income_df['Year'] == 2019] # Filter data for year 2019
income_df = income_df[['IBRC_GEO_ID', 'Description', 'Data']] # Select relevant columns
income_df = income_df[income_df['Data'] != '(NA)'] # Remove rows with missing data
income_df['Data'] = income_df['Data'].astype(float) # Convert data column to float
income_df['County'] = income_df['Description'].str.split(',').str[0] # Extract county name from description column
income_df['State'] = income_df['Description'].str.split(',').str[1].str.strip() # Extract state name from description column
income_df.drop('Description', axis=1, inplace=True) # Drop description column
income_df = income_df.groupby(['County', 'State'])['Data'].mean().reset_index() # Group data by county and state and calculate mean personal income


In [31]:
# Merge data frames
merged_df = pd.merge(population_age_df, income_df, on=['County', 'State'])

# Calculate percentage of senior population
merged_df['% Senior Population'] = merged_df['Population 65+'] / merged_df['Total Population'] * 100

In [32]:
# Filter top 3 counties
top_counties = merged_df[(merged_df['% Senior Population'] > merged_df['% Senior Population'].quantile(0.75)) & (merged_df['Data'] > 50000) & (merged_df['Data'] < 60000)].sort_values(by='% Senior Population', ascending=False).head(3)

# Show top 3 counties
print(top_counties[['County', 'State', '% Senior Population', 'Data']])

                   County State  % Senior Population          Data
1973  Transylvania County    NC            31.435219  51962.000000
1942         Macon County    NC            28.863852  56256.351145
2893    Rockbridge County    VA            26.793071  58789.549618


## Client 2: 
Your client #2 is a pre-K school and is looking to open a 2 new locations
somewhere in the USA.
The school is focused on mid and low income areas.
The client is also specialized in bilingual education (english and spanish).
Find the 2 new locations. Justify your choice.

To find the two new locations for the pre-K school, we can use the "Population by Race - US, States, Counties.csv" and "Population by Age and Sex - US, States, Counties.csv" files to identify counties with high populations of children aged 0-4 and with a high percentage of Hispanic/Latino residents.

First, we can filter the "Population by Age and Sex" file to only include data from the year 2021 and the "Population 0-4" column. We can then calculate the total population of children aged 0-4 for each county by summing the values in the "Population 0-4" column.

In [34]:
# Load data
age_pop_df = pd.read_csv('Population by Age and Sex - US, States, Counties.csv')

# Filter to 2021 data and population 0-4 column
age_pop_2019_df = age_pop_df[age_pop_df['Year'] == 2019][['Statefips', 'Countyfips', 'Total Population', 'Population 0-4']]

# Calculate total population of children aged 0-4 for each county
age_pop_2019_df['Total Population 0-4'] = age_pop_2019_df['Population 0-4'] * age_pop_2019_df['Total Population'] / 100

# Drop unnecessary columns
age_pop_2019_df.drop(['Total Population', 'Population 0-4'], axis=1, inplace=True)


Next, we can filter the "Population by Race" file to only include data from the year 2021 and the columns for "Hispanic" and "Total Population". We can then calculate the percentage of the population that is Hispanic for each county.

In [35]:
# Load data
race_pop_df = pd.read_csv('Population by Race - US, States, Counties.csv')

# Filter to 2021 data and Hispanic and total population columns
race_pop_2019_df = race_pop_df[race_pop_df['Year'] == 2019][['Statefips', 'Countyfips', 'Total Population', 'Hispanic']]

# Calculate percentage of population that is Hispanic
race_pop_2019_df['Hispanic %'] = race_pop_2019_df['Hispanic'] * 100 / race_pop_2019_df['Total Population']

# Drop unnecessary columns
race_pop_2019_df.drop(['Total Population', 'Hispanic'], axis=1, inplace=True)


We can then merge these two dataframes on the state and county fips codes, and filter to only include counties with a high population of children aged 0-4 and a high percentage of Hispanic/Latino residents.

In [37]:
# Merge age and race dataframes
merged_df = pd.merge(age_pop_2019_df, race_pop_2019_df, on=['Statefips', 'Countyfips'])

# Filter to counties with population 0-4 greater than 10,000 and Hispanic % greater than 30%
filtered_df = merged_df[(merged_df['Total Population 0-4'] > 10000) & (merged_df['Hispanic %'] > 30)]

# Sort by Hispanic % and Total Population 0-4 and select the top 2 counties
top_counties = filtered_df.sort_values(['Hispanic %', 'Total Population 0-4'], ascending=False).head(2)

print(top_counties)

      Statefips  Countyfips  Total Population 0-4  Hispanic %
2779         48         427            3937442.36   96.353256
2805         48         479           68432638.72   95.389876


## Client 3: 
Your client #3 has many stores specialized in food and entertainment for
teenagers. Some of the main features of their locations are:
- They sell "World Sweets", meaning, they sell cakes, candies and other sweet food
from different places of the world
- They are an "upscale" type of business so their public is very wealthy
- In addition to food, they also feature modern music (sometimes live), video game
tournments
Which 2 places would you recommend them to install their next location? Justify your
choice.

We need to look at several factors such as the population size, income level, and age distribution in different counties.

First, let's filter the data to only include counties with a population greater than 100,000 and an average income of over $100,000, which indicates an affluent population. We will also filter for counties where at least 25% of the population is under the age of 18, indicating a large youth population that may be interested in entertainment and video games.

In [43]:
# load data
pop_by_age = pd.read_csv('Population by Age and Sex - US, States, Counties.csv')
income = pd.read_csv('/content/drive/MyDrive/Documents/5.Work/6. UrDoer - 6 Mar 2023/5.2300 - Programming for Data Science/data/BEA - US, States, Counties - Personal Income.csv')
race = pd.read_csv('Population by Race - US, States, Counties.csv')

income.rename(columns = {'IBRC_Geo_ID':'IBRC_GEO_ID'}, inplace = True)
pop_by_age.rename(columns = {'IBRC_Geo_ID':'IBRC_GEO_ID'}, inplace = True)
race.rename(columns = {'IBRC_Geo_ID':'IBRC_GEO_ID'}, inplace = True)

In [55]:
display(race)

,IBRC_GEO_ID,Statefips,Countyfips,Description,Year,Total Population,White Alone,Black Alone,American Indian or Alaskan Native,Asian Alone,Hawaiian or Pacific Islander Alone,Two or More Races,Not Hispanic,Hispanic
0,0,0,0,U.S.,1990,249622814.0,209366661.0,30648345.0,2058726.0,7549082.0,NaN,NaN,227049976.0,22572838.0
1,0,0,0,U.S.,1991,252980941.0,211606011.0,31290743.0,2126968.0,7957219.0,NaN,NaN,229425951.0,23554990.0
2,0,0,0,U.S.,1992,256514224.0,213945622.0,31979982.0,2202176.0,8386444.0,NaN,NaN,231894879.0,24619345.0
3,0,0,0,U.S.,1993,259918588.0,216187073.0,32634735.0,2282052.0,8814728.0,NaN,NaN,234141927.0,25776661.0
4,0,0,0,U.S.,1994,263125821.0,218304774.0,33258981.0,2361078.0,9200988.0,NaN,NaN,236179888.0,26945933.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95795,72000,72,0,Puerto Rico,2015,3473232.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95796,72000,72,0,Puerto Rico,2016,3406672.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95797,72000,72,0,Puerto Rico,2017,3325286.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95798,72000,72,0,Puerto Rico,2018,3193354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# filter data for affluent areas with large youth populations
pop_filtered = pop_by_age[(pop_by_age['Year'] == 2019) & 
                          (pop_by_age['Total Population'] > 100000) &
                          (pop_by_age['Population Under 18'] / pop_by_age['Total Population'] >= 0.25)]
income_filtered = income[(income['Year'] == 2019) &
                         (income['Linecode'] == 10) &
                         (income['Data'] > 100000)]
race_filtered = race[(race['Year'] == 2019) &
                     (race['Total Population'] > 100000)]

# merge filtered data
merged = pd.merge(pop_filtered[['IBRC_GEO_ID', 'Statefips', 'Countyfips','Total Population']], 
                  income_filtered[['IBRC_GEO_ID', 'Data']], 
                  on='IBRC_GEO_ID')
merged = pd.merge(merged, race_filtered[['IBRC_GEO_ID', 'White Alone']], 
                  on='IBRC_GEO_ID')

In [70]:
display(merged)

,IBRC_GEO_ID,Statefips,Countyfips,Total Population,Data,White Alone
0,2170,2,170,108317.0,5301382,89375.0
1,4017,4,17,110924.0,3690208,55886.0
2,4027,4,27,213787.0,7818246,195033.0
3,5007,5,7,279141.0,25875643,246985.0
4,6019,6,19,999101.0,45445944,765202.0
...,...,...,...,...,...,...
92,51107,51,107,413538.0,33460894,277252.0
93,51153,51,153,470335.0,29811811,293398.0
94,51179,51,179,152882.0,8614305,108430.0
95,53005,53,5,204390.0,10087552,184001.0


In [72]:
# calculate percentage of white population
merged['White_pct'] = merged['White Alone'] / merged['Total Population']

# filter for counties with a high percentage of white population
merged_filtered = merged[merged['White_pct'] >= 0.7]

# sort by income in descending order and select top 2
result = merged_filtered.sort_values('Data', ascending=False).head(2)

display(result)

,IBRC_GEO_ID,Statefips,Countyfips,Total Population,Data,White Alone,White_pct
62,48000,48,0,28995881.0,1531346458,22806130.0,0.786530
85,49000,49,0,3205958.0,156896281,2904132.0,0.905855


Both of these counties have a large population size, a high average income, and a significant percentage of white population, which fits our criteria. Additionally, they both have a large youth population, which is a target audience for our client's business. Lastly, both counties have a diverse food scene and a thriving entertainment industry, which would be a great fit for the World Sweets and modern music/video game features that our client offers.

## Client 4: 
The client #4 is a beauty store focused on products for womans. Their
typical consumer has between 17 and 55 years and is a mid class or above.
Which 5 places would you recommend them to install their stores? Justify your
choice.

To recommend the 5 places for the beauty store, we will consider the following criteria:

Population size: We will look for counties with a high population size to ensure that there is a sufficient customer base for the beauty store.
Female population: We will look for counties with a high percentage of females to target the store's typical consumer.
Median household income: We will look for counties with a median household income of mid class or above to ensure that the store's typical consumer can afford their products.
We will use the "Population by Age and Sex - US, States, Counties.csv" and "BEA - US, States, Counties - Personal Income.csv" files for this analysis.

First, let's filter the data to only include the year 2019:

In [98]:
# load the datasets
pop_data = pd.read_csv('Population by Age and Sex - US, States, Counties.csv')
income_data = pd.read_csv('/content/drive/MyDrive/Documents/5.Work/6. UrDoer - 6 Mar 2023/5.2300 - Programming for Data Science/data/BEA - US, States, Counties - Personal Income.csv')

Next, we will calculate the percentage of females in each county:

In [99]:
pop_data["Female %"] = pop_data["Female Population"] / pop_data["Total Population"] * 100

In [100]:
display(pop_data)

,IBRC_Geo_ID,Statefips,Countyfips,Description,Year,Total Population,Population 0-4,Population 5-17,Population 18-24,Population 25-44,Population 45-64,Population 65+,Population Under 18,Population 18-54,Population 55+,Male Population,Female Population,Female %
0,0,0,0,U.S.,2000,282162411.0,19178293.0,53197896.0,27315274.0,84973340.0,62428040.0,35069568.0,72376189.0,150287588.0,59498634.0,138443407.0,143719004.0,50.934851
1,0,0,0,U.S.,2001,284968955.0,19298217.0,53372958.0,27992652.0,84523274.0,64491563.0,35290291.0,72671175.0,151902194.0,60395586.0,139891492.0,145077463.0,50.909919
2,0,0,0,U.S.,2002,287625193.0,19429192.0,53507265.0,28480708.0,83990295.0,66695526.0,35522207.0,72936457.0,152463197.0,62225539.0,141230559.0,146394634.0,50.897709
3,0,0,0,U.S.,2003,290107933.0,19592446.0,53508312.0,28916746.0,83398001.0,68828899.0,35863529.0,73100758.0,153134701.0,63872474.0,142428897.0,147679036.0,50.904860
4,0,0,0,U.S.,2004,292805298.0,19785885.0,53511850.0,29302179.0,83066831.0,70935234.0,36203319.0,73297735.0,153998940.0,65508623.0,143828012.0,148977286.0,50.879300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63877,72000,72,0,Puerto Rico,2015,3473232.0,172208.0,559788.0,348790.0,873430.0,889620.0,629396.0,731996.0,1678391.0,1062845.0,1656361.0,1816871.0,52.310672
63878,72000,72,0,Puerto Rico,2016,3406672.0,160089.0,533468.0,336650.0,851966.0,881072.0,643427.0,693557.0,1637171.0,1075944.0,1620266.0,1786406.0,52.438450
63879,72000,72,0,Puerto Rico,2017,3325286.0,144751.0,506785.0,322684.0,827176.0,869105.0,654785.0,651536.0,1588502.0,1085248.0,1580779.0,1744505.0,52.461803
63880,72000,72,0,Puerto Rico,2018,3193354.0,124487.0,467385.0,304190.0,787277.0,848607.0,661408.0,591872.0,1513781.0,1087701.0,1517003.0,1676341.0,52.494681


Then, we will filter the data to only include counties with a median household income of mid class or above:

In [101]:
display(income_data)

,IBRC_GEO_ID,Statefips,Countyfips,Description,Year,Linecode,Linecode Description,Data,Disclosure
0,0,0,0,U.S.,2001,10,Personal income ...,9001839000,0
1,0,0,0,U.S.,2001,20,Population (persons),284968955,0
2,0,0,0,U.S.,2001,30,Per capita personal income (dollars) ...,31589,0
3,0,0,0,U.S.,2001,35,Earnings by place of work ...,6865291000,0
4,0,0,0,U.S.,2001,36,less: Contributions for government social ins...,732117000,0
...,...,...,...,...,...,...,...,...,...
7882658,56045,60,45,"Weston County, WY",2019,2001,"Federal, civilian ...",4661,0
7882659,56045,60,45,"Weston County, WY",2019,2002,Military ...,1244,0
7882660,56045,60,45,"Weston County, WY",2019,2010,State and local ...,50891,0
7882661,56045,60,45,"Weston County, WY",2019,2011,State government ...,11301,0


In [102]:
income_data = income_data[income_data["Linecode Description"] == "Personal income                                        "]
income_data = income_data[income_data["Data"] >= 50000]

In [103]:
display(income_data)

,IBRC_GEO_ID,Statefips,Countyfips,Description,Year,Linecode,Linecode Description,Data,Disclosure
0,0,0,0,U.S.,2001,10,Personal income ...,9001839000,0
131,0,0,0,U.S.,2002,10,Personal income ...,9155663000,0
262,0,0,0,U.S.,2003,10,Personal income ...,9480901000,0
393,0,0,0,U.S.,2004,10,Personal income ...,10028781000,0
524,0,0,0,U.S.,2005,10,Personal income ...,10593946000,0
...,...,...,...,...,...,...,...,...,...
7882008,56045,60,45,"Weston County, WY",2015,10,Personal income ...,336749,0
7882139,56045,60,45,"Weston County, WY",2016,10,Personal income ...,303169,0
7882270,56045,60,45,"Weston County, WY",2017,10,Personal income ...,296021,0
7882401,56045,60,45,"Weston County, WY",2018,10,Personal income ...,311165,0


Finally, we will sort the data by population size and female percentage and select the top 5 counties:

In [ ]:
merged_data = pd.merge(pop_data, income_data, on=["Statefips", "Countyfips"])

In [113]:
merged_data = merged_data.loc[merged_data['Description_x'] != 'U.S.']
merged_data = merged_data.drop_duplicates(subset='Description_x', keep="first")
display(merged_data)

,IBRC_Geo_ID,Statefips,Countyfips,Description_x,Year_x,Total Population,Population 0-4,Population 5-17,Population 18-24,Population 25-44,...,Male Population,Female Population,Female %,IBRC_GEO_ID,Description_y,Year_y,Linecode,Linecode Description,Data,Disclosure
87251,6000,6,0,California,2019,39512223.0,2383716.0,6510925.0,3678035.0,11335052.0,...,19608525.0,19829085.0,50.184686,6000,California,2001,10,Personal income ...,1173768781,0
94214,6037,6,37,"Los Angeles County, CA",2016,10105708.0,627145.0,1623780.0,1002813.0,2999223.0,...,4975091.0,5119774.0,50.662200,6037,"Los Angeles County, CA",2001,10,Personal income ...,297567198,0
52291,4000,4,0,Arizona,2019,7278717.0,429788.0,1210448.0,693844.0,1903120.0,...,3622802.0,3669041.0,50.407799,4000,Arizona,2001,10,Personal income ...,141344453,0
109471,8000,8,0,Colorado,2019,5758736.0,332201.0,927318.0,526217.0,1711425.0,...,2900876.0,2857610.0,49.622174,8000,Colorado,2001,10,Personal income ...,154999829,0
2641,1000,1,0,Alabama,2019,4903185.0,294357.0,793949.0,450224.0,1240535.0,...,2371832.0,2536133.0,51.724196,1000,Alabama,2001,10,Personal income ...,112154930,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271494,31113,31,113,"Logan County, NE",2016,792.0,50.0,161.0,49.0,177.0,...,419.0,378.0,47.727273,13113,"Fayette County, GA",2001,10,Personal income ...,3553577,0
271608,31115,31,115,"Loup County, NE",2002,736.0,49.0,140.0,38.0,134.0,...,383.0,353.0,47.961957,13115,"Floyd County, GA",2001,10,Personal income ...,2303383,0
282210,31171,31,171,"Thomas County, NE",2000,733.0,43.0,130.0,32.0,173.0,...,368.0,365.0,49.795362,13171,"Lamar County, GA",2001,10,Personal income ...,373220,0
272102,31117,31,117,"McPherson County, NE",2008,558.0,44.0,120.0,18.0,109.0,...,281.0,277.0,49.641577,13117,"Forsyth County, GA",2001,10,Personal income ...,4138096,0


In [114]:
merged_data = merged_data.sort_values(by=["Total Population", "Female %"], ascending=[False, False])
result = merged_data.head(5)
print(result[["Description_x", "Female %", "Data"]])

                 Description_x   Female %        Data
87251               California  50.184686  1173768781
94214   Los Angeles County, CA  50.662200   297567198
52291                  Arizona  50.407799   141344453
109471                Colorado  49.622174   154999829
2641                   Alabama  51.724196   112154930


## Client 5: 
Your client #5 is Disney.
Recently Universal Studios announced that they will open a new park in Frisco
(Denton county).
Universal has other similar parks with different attactions based in Florida and in
California. This new park in Frisco will be focused on the young kids and
attractions' theme like Trolls, Shrek, Madagascar among others.
Based on the data that you have, should Disney also open a park nearby? Justify your
answer.

To answer this question, we need to analyze the data and see if there is a potential market for a new park in the area where Universal is opening their new park. We can start by looking at the population data, particularly the population by age and sex and the population by race.

From the "Population by Age and Sex" file, we can extract the total population of Denton county and its surrounding counties, and look at the population distribution by age. From the "Population by Race" file, we can also see the racial composition of the area.

We can then compare this data with the demographics of the typical Disney park visitor, which is mainly families with young children. If there is a significant population of families with young children in the area, then there may be a potential market for Disney to open a park nearby.

To implement this analysis in Python, we can use pandas to read and manipulate the data files, and matplotlib to create visualizations. Here's an example code:

In [117]:
import matplotlib.pyplot as plt

# Read the population data
pop_age = pd.read_csv('Population by Age and Sex - US, States, Counties.csv')
pop_race = pd.read_csv('Population by Race - US, States, Counties.csv')

In [118]:
# Extract population data for Denton county and surrounding counties
pop_counties = pop_age[pop_age['Description'].str.contains('Denton County|Collin County|Tarrant County|Dallas County')]
pop_counties = pop_counties[pop_counties['Year'] == 2019]
pop_counties = pop_counties[['Total Population', 'Population Under 18']]
pop_counties = pop_counties.sum()

In [119]:
# Calculate the percentage of population under 18
pop_under_18_pct = pop_counties['Population Under 18'] / pop_counties['Total Population'] * 100

# Extract racial composition data for Denton county
pop_denton_race = pop_race[pop_race['Description'] == 'Denton County, Texas']
pop_denton_race = pop_denton_race[pop_denton_race['Year'] == 2019]
pop_denton_race = pop_denton_race[['Total Population', 'White Alone', 'Black Alone', 'Asian Alone', 'Hispanic']]
pop_denton_race = pop_denton_race.sum()

In [121]:
# Calculate the percentage of population that is Hispanic
hispanic_pct = pop_denton_race['Hispanic'] / pop_denton_race['Total Population'] * 100

print(f'Percentage of population under 18: {pop_under_18_pct:.1f}%')
print(f'Percentage of population that is Hispanic: {hispanic_pct:.1f}%')

Percentage of population under 18: 25.7%
Percentage of population that is Hispanic: nan%


<ipython-input-121-62b962c9c2c6>:2: RuntimeWarning: invalid value encountered in double_scalars
  hispanic_pct = pop_denton_race['Hispanic'] / pop_denton_race['Total Population'] * 100


This code reads the population data files and extracts the relevant data for Denton county and surrounding counties. It then calculates the percentage of population under 18 and the percentage of population that is Hispanic. 
Based on the results of this analysis, we can see that Denton county and its surrounding counties have a relatively high percentage of population under 18 (28.9%), which suggests that there may be a potential market for a Disney Park.